In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

C:\Users\mibob_000\AppData\Roaming\Python\Python35\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


#### Import data

In [3]:
BATCH_SIZE = 6

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


#### Show some images and labels

In [4]:
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(BATCH_SIZE)))

  dog   cat  frog  ship truck  bird


#### Net definition

In [346]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#### Resnet definition 

In [15]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

    
NUM_LAYERS = 8
WIDTH = 64
NUM_CLASSES = 10
IMAGE_DIMENSIONS = 32*32*3
CONV_WIDTH = 6

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = 'cpu'
print(device)

class AntiSymmetric(nn.Module):
    def __init__(self):
        super(AntiSymmetric, self).__init__()
        self.conv = nn.Conv2d(CONV_WIDTH, CONV_WIDTH, 5, padding=(2, 2))
        
    def forward(self, x):
        self.weight = self.weight - self.weight.t()
        self.conv(x)
        
        

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.fcf = nn.Linear(CONV_WIDTH*32*32, NUM_CLASSES)
        self.convs = nn.Conv2d(3, CONV_WIDTH, 5, padding=(2, 2))
        self.conv = nn.ModuleList()
        for i in range(NUM_LAYERS):
            self.conv.append(nn.Conv2d(CONV_WIDTH, CONV_WIDTH, 5, padding=(2, 2)))
        
        def init_weights(m):
            if type(m) == nn.Conv2d:
                torch.nn.init.xavier_normal_(m.weight, gain=0.01)
                m.bias.data.fill_(0.01)
        self.conv.apply(init_weights)
    
    def forward(self, x):
        x = F.relu(self.convs(x))
        for i in range(NUM_LAYERS):
            x = x + F.relu(self.conv[i](x))
        x = x.view(-1, CONV_WIDTH*32*32)
        x = F.relu(self.fcf(x))
        return x
    

net = ResNet()
print(net.to(device))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.Adam(net.parameters()) #(net.parameters(), lr=0.001, momentum=0.9)

cuda:0
ResNet(
  (fcf): Linear(in_features=6144, out_features=10, bias=True)
  (convs): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv): ModuleList(
    (0): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (6): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (7): Conv2d(6, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
)


tensor([[-0.0458,  0.0831, -0.1415,  ...,  0.0244,  0.0387, -0.0194],
        [ 0.0636,  0.0182, -0.0476,  ..., -0.0229,  0.0011,  0.0812],
        [ 0.0161,  0.0438,  0.1046,  ...,  0.0475, -0.0530,  0.0425],
        ...,
        [-0.0177, -0.0380,  0.2548,  ...,  0.0719,  0.2149,  0.0162],
        [-0.0560, -0.1308,  0.0105,  ..., -0.1914, -0.1581, -0.1425],
        [-0.1942,  0.0880, -0.0641,  ...,  0.0870, -0.1399, -0.0789]],
       device='cuda:0', grad_fn=<TBackward>)

#### Train the network

In [30]:
import time
testiter = iter(testloader)
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
for epoch in range(5):  # loop over the dataset multiple times

    start = time.time()
    running_loss = 0.0
    train_correct = 0
    train_total = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted==labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('Train accuracy: ' + str(train_correct/train_total*100) + '%')
            train_correct = 0
            train_total = 0
            correct = 0
            total = 0
            with torch.no_grad():
            #     print(time.time())
            #     for i in range(10):
            #         data = testiter.next()
                for data in testloader:
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()


            #print(time.time())
            print('Accuracy of the network on the 1000 test images: %f %%' % (
                100 * correct / total))
            
            
            print('Time: ' + str(time.time()-start))
            start = time.time()
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
import winsound

print('Finished Training')
winsound.PlaySound('./sound2.wav', winsound.SND_FILENAME)

Train accuracy: 87.5%
Accuracy of the network on the 1000 test images: 54.380000 %
Time: 31.690141916275024
[1,  2000] loss: 0.418
Train accuracy: 87.02499999999999%
Accuracy of the network on the 1000 test images: 54.220000 %
Time: 30.20831823348999
[1,  4000] loss: 0.430
Train accuracy: 86.95%
Accuracy of the network on the 1000 test images: 54.180000 %
Time: 33.39010214805603
[1,  6000] loss: 0.427
Train accuracy: 86.73333333333333%
Accuracy of the network on the 1000 test images: 54.240000 %
Time: 35.10319447517395
[1,  8000] loss: 0.436
Train accuracy: 87.20833333333333%
Accuracy of the network on the 1000 test images: 54.260000 %
Time: 34.96131467819214
[2,  2000] loss: 0.425
Train accuracy: 87.375%
Accuracy of the network on the 1000 test images: 54.280000 %
Time: 34.84831786155701
[2,  4000] loss: 0.421
Train accuracy: 87.28333333333333%
Accuracy of the network on the 1000 test images: 54.250000 %
Time: 34.72168779373169
[2,  6000] loss: 0.425
Train accuracy: 86.88333333333334%

In [325]:
correct = 0
total = 0
with torch.no_grad():
    i = 0
#     print(time.time())
#     for i in range(10):
#         data = testiter.next()
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print(time.time())
print('Accuracy of the network on the 1000 test images: %d %%' % (
    100 * correct / total))

1535118276.1108475
Accuracy of the network on the 1000 test images: 61 %


#### Accuracies of classes

In [133]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 32 %
Accuracy of   car : 51 %
Accuracy of  bird : 29 %
Accuracy of   cat : 35 %
Accuracy of  deer : 50 %
Accuracy of   dog : 14 %
Accuracy of  frog : 66 %
Accuracy of horse : 65 %
Accuracy of  ship : 67 %
Accuracy of truck : 58 %


#### Show some images and labels

In [232]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

GroundTruth:    cat  ship  ship plane


RuntimeError: Expected object of type torch.cuda.FloatTensor but found type torch.FloatTensor for argument #4 'mat1'